In [1]:
from numpy import *
from pylab import *
from numpy.random import rand
import scipy.stats

kappa=2; theta=0.0225; sigma=0.2; rho=0.5; v0=0.0225; rd=0.05; rf=0.0; T = 0.25; s0=100; K =110;N=500

########################################################################################################################################################
def trap(f,a,b,N):

    #This function calculates the iterated trapezoidrule to approximate an integral

    h = (b-a)/N
    left = 0.5*h*f(a)
    right = 0.5*h*f(b)
    mid = 0

    for i in range(1,N):
        mid = mid + h*f(a+h*i)

    integral = left+mid+right

    return integral


########################################################################################################################################################

def charfn_heston(phi,kappa,theta,sigma,rho,v0,rd,rf,T,s0,index):
    
    if index == 1:
        u = 0.5
        b = kappa - rho*sigma;
    else:
        u = -0.5
        b = kappa
    
    x = log(s0)
    a = kappa*theta; 
    d = sqrt((rho*sigma*phi*1j-b)**2-sigma**2*(2*u*phi*1j-phi**2))
    g = (b-rho*sigma*phi*1j + d)/(b-rho*sigma*phi*1j - d)
    C = (rd-rf)*phi*1j*T + a/sigma**2*((b- rho*sigma*phi*1j + d)*T - 2*log((1-g*exp(d*T))/(1-g)))
    D = (b-rho*sigma*phi*1j + d)/sigma**2*((1-exp(d*T))/(1-g*exp(d*T)))
    char_fn = exp(C + D*v0 + 1j*phi*x)

    return char_fn

########################################################################################################################################################

def quadhestoncall(kappa,theta,sigma,rho,v0,rd,rf,T,s0,K,N,uMin=0.00000001,uMax=100):

    a = uMin
    b = uMax

    def integrand1(phi,kappa=kappa,theta=theta,sigma=sigma,rho=rho,v0=v0,rd=rd,rf=rf,T=T,s0=s0,K=K,index=1):
        
        integrand1 = ((exp(-1j*phi*log(K))*charfn_heston(phi,kappa,theta,sigma,rho,v0,rd,rf,T,s0,index))/(1j*phi)).real
        
        return integrand1

    def integrand2(phi,kappa=kappa,theta=theta,sigma=sigma,rho=rho,v0=v0,rd=rd,rf=rf,T=T,s0=s0,K=K,index=2):

        integrand2 = ((exp(-1j*phi*log(K))*charfn_heston(phi,kappa,theta,sigma,rho,v0,rd,rf,T,s0,index))/(1j*phi)).real
        
        return integrand2
    

    p1 = 0.5 + (1/pi)*trap(integrand1,a,b,N)
    p2 = 0.5 + (1/pi)*trap(integrand2,a,b,N)

    callprice=s0*exp(-rf*T)*p1 - K*exp(-rd*T)*p2
    
    return callprice

########################################################################################################################################################


In [2]:
quadhestoncall(kappa,theta,sigma,rho,v0,rd,rf,T,s0,K,N,uMin=0.00000001,uMax=100)

0.6614034902557488